# 分析目的

- 探索票房与导演、主演之间的关系
- 评分趋势
- 电影产量与年份之间的关系
- 电影类型与票房的关系

## 数据预处理

In [175]:
import pandas as pd
import numpy as np
import matplotlib as mpt
from matplotlib import pyplot as plt
import seaborn as sns

movie_row_data = pd.read_csv(r'../20191016_IMDB Movies_MetaData.csv', sep=',')

### 数据预览

In [44]:
movie_row_data.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


### 缺失值处理

In [45]:
movie_row_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5043 entries, 0 to 5042
Data columns (total 28 columns):
color                        5024 non-null object
director_name                4939 non-null object
num_critic_for_reviews       4993 non-null float64
duration                     5028 non-null float64
director_facebook_likes      4939 non-null float64
actor_3_facebook_likes       5020 non-null float64
actor_2_name                 5030 non-null object
actor_1_facebook_likes       5036 non-null float64
gross                        4159 non-null float64
genres                       5043 non-null object
actor_1_name                 5036 non-null object
movie_title                  5043 non-null object
num_voted_users              5043 non-null int64
cast_total_facebook_likes    5043 non-null int64
actor_3_name                 5020 non-null object
facenumber_in_poster         5030 non-null float64
plot_keywords                4890 non-null object
movie_imdb_link              5043 non-

根据所得信息，了解到有缺失数据。

由于此处数据是从他处直接获得，因此无法了解缺失数据的原因，也无法对缺失数据进行探索。为了方便，在此将有缺失的数据全部删除。

In [46]:
movie_row_data.dropna(how='any',inplace=True)

In [47]:
movie_row_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3756 entries, 0 to 5042
Data columns (total 28 columns):
color                        3756 non-null object
director_name                3756 non-null object
num_critic_for_reviews       3756 non-null float64
duration                     3756 non-null float64
director_facebook_likes      3756 non-null float64
actor_3_facebook_likes       3756 non-null float64
actor_2_name                 3756 non-null object
actor_1_facebook_likes       3756 non-null float64
gross                        3756 non-null float64
genres                       3756 non-null object
actor_1_name                 3756 non-null object
movie_title                  3756 non-null object
num_voted_users              3756 non-null int64
cast_total_facebook_likes    3756 non-null int64
actor_3_name                 3756 non-null object
facenumber_in_poster         3756 non-null float64
plot_keywords                3756 non-null object
movie_imdb_link              3756 non-

### 重复值处理

In [48]:
movie_row_data.drop_duplicates(inplace=True)

In [49]:
movie_row_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3723 entries, 0 to 5042
Data columns (total 28 columns):
color                        3723 non-null object
director_name                3723 non-null object
num_critic_for_reviews       3723 non-null float64
duration                     3723 non-null float64
director_facebook_likes      3723 non-null float64
actor_3_facebook_likes       3723 non-null float64
actor_2_name                 3723 non-null object
actor_1_facebook_likes       3723 non-null float64
gross                        3723 non-null float64
genres                       3723 non-null object
actor_1_name                 3723 non-null object
movie_title                  3723 non-null object
num_voted_users              3723 non-null int64
cast_total_facebook_likes    3723 non-null int64
actor_3_name                 3723 non-null object
facenumber_in_poster         3723 non-null float64
plot_keywords                3723 non-null object
movie_imdb_link              3723 non-

In [50]:
movie_row_data.to_csv('new_output/movie_row_data.csv')

### 处理异常值

发现异常值，一般有两种方式：一是利用业务经验，查看是否存在异常数据；二是利用箱型分布或者正态分布，利用数据分布，查看是否存在异常值。

箱型分布的话，`cut`和`qcut`函数可以实现分箱离散化。

业务经验其实就是对数据的了解程度。比方说，简单些，该电影数据中‘title_year’出现‘2020’这个数字，这就是异常数据。如果对数据没有相当了解，比较难以依靠业务经验发现异常值。

另一个就是绘制箱型图或者正态分布图，查看数据分布是否有明显异常。此处在数据分析时再详细使用。

### 数据类型转换

做不同的数据分析可能会需要不同的数据类型，而数据在某一刻只能有一种数据类型，因此，在做具体数据分析时，可能会按需求对数据类型进行转换。

## 实际分析

在对数据进行分析时，应该只获取目标数据，其余的数据都属于冗余数据，冗余数据会拖慢数据处理速度。

### 票房与导演、主演之间的关系

In [51]:
# 获取票房、导演、主演的数据
movie_analysis_01 = movie_row_data.loc[:,['director_name', 'gross', 'actor_1_name']]
movie_analysis_01.head()

,director_name,gross,actor_1_name
0,James Cameron,760505847.0,CCH Pounder
1,Gore Verbinski,309404152.0,Johnny Depp
2,Sam Mendes,200074175.0,Christoph Waltz
3,Christopher Nolan,448130642.0,Tom Hardy
5,Andrew Stanton,73058679.0,Daryl Sabara


In [181]:
# 查看各导演的票房总收入
movie_analysis_01.groupby(by='director_name')['gross'].sum().sort_values(ascending=False).head(10)

director_name
Steven Spielberg     4.114233e+09
Peter Jackson        2.289968e+09
Michael Bay          2.231243e+09
Tim Burton           2.071275e+09
Sam Raimi            2.049549e+09
James Cameron        1.948126e+09
Christopher Nolan    1.813228e+09
George Lucas         1.741418e+09
Robert Zemeckis      1.619309e+09
Chris Columbus       1.618708e+09
Name: gross, dtype: float64

In [53]:
# 查看各主演的票房总收入
movie_analysis_01.groupby(by='actor_1_name')['gross'].sum().sort_values(ascending=False).head(10)

actor_1_name
Johnny Depp          3.683191e+09
Harrison Ford        3.391556e+09
Tom Hanks            3.264559e+09
Tom Cruise           2.987622e+09
J.K. Simmons         2.856407e+09
Will Smith           2.762618e+09
Leonardo DiCaprio    2.640582e+09
Robert Downey Jr.    2.456990e+09
Jennifer Lawrence    2.367856e+09
Robin Williams       2.296964e+09
Name: gross, dtype: float64

### 评分趋势

In [54]:
# 查看电影与评分之间的关系
# 获取目标数据
movie_analysis_02 = movie_row_data.loc[:, ['imdb_score', 'movie_title']]
movie_analysis_02.groupby(by='imdb_score')['movie_title'].count().sort_values(ascending=False).head(10)

imdb_score
6.7    175
6.6    161
6.5    152
6.4    146
6.8    142
6.1    142
7.1    140
7.0    138
7.2    137
6.9    135
Name: movie_title, dtype: int64

In [55]:
# 查看导演与评分之间的关系
movie_analysis_03 = movie_row_data.loc[:, ['director_name', 'imdb_score']]
movie_analysis_03.groupby(by='imdb_score')['director_name'].count().sort_values(ascending=False).head()

imdb_score
6.7    175
6.6    161
6.5    152
6.4    146
6.8    142
Name: director_name, dtype: int64

### 电影产量与年份之间的关系

In [80]:
# 获取目标数据
movie_analysis_04 = movie_row_data.loc[:, ['title_year', 'movie_title']]
movie_analysis_04.groupby(by='title_year')['movie_title'].count().sort_values(ascending=False).head(10)

title_year
2006.0    188
2002.0    186
2008.0    182
2005.0    182
2009.0    181
2004.0    178
2001.0    176
2011.0    168
2010.0    167
2013.0    162
Name: movie_title, dtype: int64

In [161]:
# 可以明显看到上面的‘title_year’列的数据类型有误，本应该是日期类型，结果却是浮点类型
print("转换前的数据类型：", movie_analysis_04.title_year.dtype)

# 对该列数据进行类型转换
movie_analysis_04.title_year = movie_analysis_04.title_year.values.astype('int32')

print("转换后的数据类型：", movie_analysis_04.title_year.dtype)

转换前的数据类型： int32
转换后的数据类型： int32


In [82]:
# 重新显示结果
movie_analysis_04.groupby(by='title_year')['movie_title'].count().sort_values(ascending=False).head(10)

title_year
2006    188
2002    186
2008    182
2005    182
2009    181
2004    178
2001    176
2011    168
2010    167
2013    162
Name: movie_title, dtype: int64

### 电影类型与票房的关系

In [174]:
# 获取相关数据
movie_analysis_05 = movie_row_data.loc[:, ['genres', 'gross']]
movie_analysis_05.to_csv('new_output/genres_gross.csv')

# 重置索引
movie_analysis_05.index = range(movie_analysis_05.shape[0])

# 尝试分割目标数据
# 方法笨拙，不过这是个人目前能想到的唯一解决办法，日后熟悉使用 Python 后，说不定能写出其他的解决方案
genre_list = []
gross_list = []
for i in range(movie_analysis_05.shape[0]):
    genre_list.append(movie_analysis_05.genres[i].split("|"))
    gross_list.append(movie_analysis_05.gross[i])
    
new_genre_list = []
new_gross_list = []
for j in range(len(genre_list)):
    for k in range(len(genre_list[j])):
        new_genre_list.append(genre_list[j][k])
        new_gross_list.append(gross_list[j])
        
movie_analysis_06 = pd.DataFrame(list(zip(new_genre_list, new_gross_list)))
movie_analysis_06.columns = ['genre', 'gross']
movie_analysis_06.groupby(by='genre')['gross'].sum().sort_values(ascending=False).head(10)

genre
Adventure    7.764312e+10
Comedy       7.510514e+10
Action       7.357374e+10
Drama        7.140817e+10
Thriller     5.414381e+10
Fantasy      4.408405e+10
Family       4.120049e+10
Sci-Fi       4.093102e+10
Romance      3.869760e+10
Crime        2.767865e+10
Name: gross, dtype: float64

# 总结

## 回顾

实际上，可以写一个函数来代替上面的所有聚合代码。

In [172]:
def groupby_funtion(raw_data_name, data_name, by_column_name, calculate_column_name):
    data_name = raw_data_name.loc[:, [by_column_name, calculate_column_name]]
    return data_name.groupby(by=by_column_name)[calculate_column_name].count().sort_values(ascending=False).head(10)

## 数据处理操作相关


处理<u>缺失值</u>时，首先应该观察数据，了解到缺失数据的特点，而后再决定如何处置。

比如说，假如某行缺乏数据，但不影响数据处理，那时就可以考虑保留该行数据，使用`dropna(how="all")`，或者，`fillna("数据缺失")`。

又或者如果缺失的数据属于数字类型，那时便可以考虑使用平均值、前值、后值或者 0 填充。

处理<u>重复值</u>时，可以考虑按照多列重复来删除数据。

<u>异常值，就是不符合预期的值。</u>发现异常值的方式，大致有两种。但这两种其实都离不开对业务的理解。比如说，假如数据符合正态分布或者箱型分布，但是一看，明显和实际业务不符，此时肯定要深究。数据分析离不开对业务的理解。

而对异常值的处理，可以删除、替换，或者研究异常值出现的原因。根据个人这些天对数据分析的了解，或许**研究异常值**可能会是最佳选择。数据只是行为的结果，出现异常数据，那么很有可能是因为行为有异。

另外，数据处理后，注意索引值，如果需要重新序列化索引，那就必须重新序列化索引。

## 其他

实际上这个数据分析案例完全不合。

一是只分析单一变量之间的分析，未考虑到其他变量。比如说导演和票房之间的关系，未考虑到预算、电影数量等其他因素。因此，不能说某导演票房高，因此他就每部都卖座。从数据本身是能看到不少东西，但数据看不到所有。典型的如大鹏，大鹏的第一部电影《煎饼侠》的票房很高，但是第二部《缝纫机乐队》就扑街。还有就是吴京的《战狼》系列电影，谁能想到后面情势一变，使《战狼 2》成为中国历史票房最高的电影呢？这些从数据怎么看？看不到的。数据都是历史，如果能根据数据来预测，那就非常棒。可惜，从个人现在所自学的东西来看，绝大部分都是教人怎么处理数据（历史），而不是根据数据来结合现实（业务）来预测收益（未来）。

二是数据命名混乱，或者说，缺乏对数据集的介绍。

三是数据量太小，导致分析结果无用。

该数据分析案例纯粹就是用来练习各种数据分析基本操作，无其他太大用处。

所谓“数据分析”，个人理解就是利用数据（历史），结合实际（业务），使用合理工具（算法、模型？），对未来（收益）进行预测。

数据是行为的结果，而不是原因。我想，还是要根据数据分析的结果去找原因，找到原因后才能指导业务，进而获得收益。

今后的个人重点或许会是算法与业务。